# 让字典的键值序列支持整数下标运算

Python 3.6的字典能保持键的插入顺序。而其`keys()`, `values()`和`items()`方法返回的都是字典的视图对象，这些对象不支持下标运算。所以下面的程序抛出异常。

In [24]:
d = {"a":1, "b":2, "c":3}
k = d.keys()
k[1]

TypeError: 'dict_keys' object does not support indexing

为了获取指定下标的元素，可以使用`itertools.islice()`和`next()`函数。

In [27]:
from itertools import islice
next(islice(k, 1, None))

'b'

为了让`dict_keys()`对象支持下标运算，需要将其对应的类型结构体`PyTypeObject`的`tp_as_sequence`字段所指向的`PySequenceMethods`结构体的`sq_item`字段设置进行下标运算的函数，该函数的原型如下：

`PyObject * (*)(PyObject *a, Py_ssize_t i)`

下面使用Cython编写该函数，由于该函数只能被C语言调用，因此使用`cdef`定义它，并将其地址保存到`addr_sq_item`变量中。

In [47]:
%load_ext cython

In [60]:
%%cython
from itertools import islice        

cdef sq_item(object dv, ssize_t index):
    if index < 0:
        index += len(dv)
    try:
        return next(islice(dv, index, None))
    except StopIteration:
        raise IndexError("Out of range")

addr_sq_item = <long>&(sq_item)

下面的`cffi_build()`使用`cffi`编译模块并载入模块，模块的文件名由编译的内容决定。

In [78]:
def import_file(name, filename):
    import importlib.util
    spec = importlib.util.spec_from_file_location(name, filename)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def cffi_build(cdef, source, disable_py_limited_api=True, force=False):
    import cffi
    import time
    import hashlib
    import imp
    from os import path
    from distutils.sysconfig import get_config_var
    tmpdir = path.abspath(".\\__pycache__")
    suffix = get_config_var('EXT_SUFFIX')
    key = cdef + source
    if force:
        key += time.time()
    filename = "_cffi_" + hashlib.md5(str(key).encode("utf-8")).hexdigest()
    full_filename = path.join(tmpdir, filename + suffix)
    if not path.exists(full_filename):
        ffi = cffi.FFI()    
        extra_compile_args = ["-D_CFFI_USE_EMBEDDING"] if disable_py_limited_api else []
        ffi.set_source(filename, source, extra_compile_args=extra_compile_args)
        ffi.cdef(cdef)
        full_filename = ffi.compile(tmpdir=tmpdir)
    return import_file(filename, path.join(tmpdir, full_filename))

下面的`set_sq_item()`设置`type_addr`类的`sq_item`插槽：

In [79]:
source = """
void set_sq_item(uintptr_t type_addr, uintptr_t func_addr)
{
    uintptr_t sequence_slot_addr = type_addr + offsetof(PyTypeObject, tp_as_sequence);
    uintptr_t sq_item_addr = *(uintptr_t *)(sequence_slot_addr) + offsetof(PySequenceMethods, sq_item);
    *(uintptr_t *)sq_item_addr = func_addr;
}
"""

cdef = """
void set_sq_item(uintptr_t type_addr, uintptr_t func_addr);
"""

m = cffi_build(cdef, source)

我们使用`set_sq_item()`设置`dict_keys`类的`sq_item`插槽：

In [80]:
m.lib.set_sq_item(id(type(k)), addr_sq_item)

于是可以使用下标运算了：

In [81]:
k[1]

'b'

下面为`values()`和`items()`所返回的对象的类`dict_values`和`dict_items`添加`sq_item`插槽：

In [82]:
v = d.values()
m.lib.set_sq_item(id(type(v)), addr_sq_item)
v[-1]

3

In [83]:
items = d.items()
m.lib.set_sq_item(id(type(items)), addr_sq_item)
items[-2]

('b', 2)